In [2]:
import sys
import io
import numpy as np
import string
import os
import matplotlib
import matplotlib.pyplot as plt
import pylab
from scipy.optimize import linprog
from scipy.stats import gaussian_kde
import math
from Bidder import Bidder
from Auction import Auction
from Slots import Slots

In [3]:
def runAndPrint(numbid, numitem, valueGenFunc, truthOrRandom, repeatTimes):
    auction = Auction(numbid, numitem, valueGenFunc, truthOrRandom)
    SWVCGseq, RVCGseq, SWGSPseq, RGSPseq  = [], [], [], []
    for i in range(repeatTimes):
        # auction = Auction(numbid, numitem, truthOrRandom)
        auction.clearData()

        auction.executeVCG()
        swVCG = auction.SocialWelfare
        rVCG = auction.revenue
        SWVCGseq = np.append(SWVCGseq, swVCG)
        RVCGseq = np.append(RVCGseq, rVCG)

        auction.executeGSP()
        swGSP = auction.SocialWelfare
        rGSP = auction.revenue
        SWGSPseq = np.append(SWGSPseq, swGSP)
        RGSPseq = np.append(RGSPseq, rGSP)
    SWVCG = np.mean(SWVCGseq)
    RVCG = np.mean(RVCGseq)
    SWGSP = np.mean(SWGSPseq) 
    RGSP = np.mean(RGSPseq)
    print("seq: RVCG[0] = " + str(RVCGseq[0]) + ", RVCG[n-1] = " + str(RVCGseq[repeatTimes-1]))
    print("seq: RGSP[0] = " + str(RGSPseq[0]) + ", RGSP[n-1] = " + str(RGSPseq[repeatTimes-1]))
    return [SWVCG, RVCG, SWGSP, RGSP]

In [4]:
if __name__ == '__main__':
    valueGenFunc, truthOrRandom = ['uniform', 'uniform-BsmallthanV', 'normal'], ['random', 'truth']
    lenValueGenFunc = len(valueGenFunc)
    SWVCG = [[[] for i in range(lenValueGenFunc)] for j in range(2)]#[[[]]*(lenValueGenFunc)]*2
    SWGSP = [[[] for i in range(lenValueGenFunc)] for j in range(2)]
    RVCG = [[[] for i in range(lenValueGenFunc)] for j in range(2)]
    RGSP = [[[] for i in range(lenValueGenFunc)] for j in range(2)]

    for j in range(2):
        for num in list(range(1,21)):
            for i in range(2):
                print("The number of Bidders is " + str(num) + ", and the number of Items is " + str(num) + ", " + truthOrRandom[i])
                [swVCG, rVCG, swGSP, rGSP] = runAndPrint(num, num, valueGenFunc[j],truthOrRandom[i], 10000)
                SWVCG[j][i] = np.append(SWVCG[j][i], swVCG)
                RVCG[j][i] = np.append(RVCG[j][i], rVCG)
                SWGSP[j][i] = np.append(SWGSP[j][i], swGSP)
                RGSP[j][i] = np.append(RGSP[j][i], rGSP)
                print("SWVCG = " + str(swVCG) + ", SWGSP = " + str(swGSP) + ", RVCG = " + str(rVCG) + ", RGSP = " + str(rGSP))

    x_axis=[i for i in range(1,21)]
        
    # print(SWVCG[0][0])
    Revenue = io.open("Revenue.txt","w")
    SocialWelfare = io.open("SocialWelfare.txt","w")
    for j in range(2):
        for i in range(2):
            SocialWelfare.writelines("SWVCG" + valueGenFunc[j] + " " + truthOrRandom[i] +str(SWVCG[j][i]) + "\n")
            SocialWelfare.writelines("SWGSP" + valueGenFunc[j] + " " + truthOrRandom[i] +str(SWGSP[j][i]) + "\n")
            Revenue.writelines("RVCG" + valueGenFunc[j] + " " + truthOrRandom[i] +str(RVCG[j][i]) + "\n")
            Revenue.writelines("RGSP" + valueGenFunc[j] + " " + truthOrRandom[i] +str(RGSP[j][i]) + "\n")
    Revenue.close()
    SocialWelfare.close()
    plt.figure(1)
    plt.xlabel("The number of bidder and slot")
    plt.ylabel("social welfare")
    plt.title("social welfare with different number of bidder and slot")
    plt.plot(x_axis,SWVCG[0][0],label = truthOrRandom[0] + ' VCG')
    plt.plot(x_axis,SWGSP[0][0],label = truthOrRandom[0] + ' GSP')
    plt.plot(x_axis,SWVCG[0][1],label = truthOrRandom[1] + ' VCG')
    plt.plot(x_axis,SWGSP[0][1],label = truthOrRandom[1] + ' GSP')
    plt.plot(x_axis,SWVCG[1][0],label = "b<v " + truthOrRandom[0] + ' VCG')
    plt.plot(x_axis,SWGSP[1][0],label = "b<v " + truthOrRandom[0] + ' GSP')
    plt.plot(x_axis,SWVCG[1][1],label = "b<v " + truthOrRandom[1] + ' VCG')
    plt.plot(x_axis,SWGSP[1][1],label = "b<v " + truthOrRandom[1] + ' GSP')
    plt.legend()
    plt.savefig("social welfare.png")

    plt.figure(2)
    plt.xlabel("The number of bidder and slot")
    plt.ylabel("revenue")
    plt.title("revenue with different number of bidder and slot")
    plt.plot(x_axis,RVCG[0][0],label = truthOrRandom[0] + ' VCG')
    plt.plot(x_axis,RGSP[0][0],label = truthOrRandom[0] + ' GSP')
    plt.plot(x_axis,RVCG[0][1],label = truthOrRandom[1] + ' VCG')
    plt.plot(x_axis,RGSP[0][1],label = truthOrRandom[1] + ' GSP')
    plt.plot(x_axis,RVCG[1][0],label = "b<v " + truthOrRandom[0] + ' VCG')
    plt.plot(x_axis,RGSP[1][0],label = "b<v " + truthOrRandom[0] + ' GSP')
    plt.plot(x_axis,RVCG[1][1],label = "b<v " + truthOrRandom[1] + ' VCG')
    plt.plot(x_axis,RGSP[1][1],label = "b<v " + truthOrRandom[1] + ' GSP')
    plt.legend()
    plt.savefig("revenue.png")

The number of Bidders is 1, and the number of Items is 1, random
seq: RVCG[0] = 0.0, RVCG[n-1] = 0.0
seq: RGSP[0] = 0.0, RGSP[n-1] = 0.0
SWVCG = 2.50060974636, SWGSP = 2.50060974636, RVCG = 0.0, RGSP = 0.0
The number of Bidders is 1, and the number of Items is 1, truth
seq: RVCG[0] = 0.0, RVCG[n-1] = 0.0
seq: RGSP[0] = 0.0, RGSP[n-1] = 0.0
SWVCG = 2.53245821068, SWGSP = 2.53245821068, RVCG = 0.0, RGSP = 0.0
The number of Bidders is 2, and the number of Items is 2, random
seq: RVCG[0] = 0.0, RVCG[n-1] = 0.0
seq: RGSP[0] = 2.11795108932, RGSP[n-1] = 0.204248592068
SWVCG = 5.08007713934, SWGSP = 5.08007713934, RVCG = 0.0, RGSP = 2.24194690491
The number of Bidders is 2, and the number of Items is 2, truth
seq: RVCG[0] = 0.0, RVCG[n-1] = 0.0
seq: RGSP[0] = 1.87807912358, RGSP[n-1] = 1.48665536103
SWVCG = 5.55114267254, SWGSP = 5.55114267254, RVCG = 0.0, RGSP = 2.22933230797
The number of Bidders is 3, and the number of Items is 3, random
seq: RVCG[0] = 0.0, RVCG[n-1] = 1.21882968685e-05
se

seq: RVCG[0] = 10.4524506452, RVCG[n-1] = 16.3768180612
seq: RGSP[0] = 57.3255786977, RGSP[n-1] = 55.0830166729
SWVCG = 42.5167891977, SWGSP = 42.5167891977, RVCG = 14.8049333134, RGSP = 50.3612357141
The number of Bidders is 17, and the number of Items is 17, truth
seq: RVCG[0] = 17.0125259273, RVCG[n-1] = 8.48466607479
seq: RGSP[0] = 53.2726040124, RGSP[n-1] = 49.0922688918
SWVCG = 55.1741961923, SWGSP = 55.1741961923, RVCG = 8.67350462077, RGSP = 50.4552927787
The number of Bidders is 18, and the number of Items is 18, random
seq: RVCG[0] = 17.7552397022, RVCG[n-1] = 8.68076840073
seq: RGSP[0] = 69.4841392196, RGSP[n-1] = 53.7992697407
SWVCG = 45.1460238982, SWGSP = 45.1460238982, RVCG = 16.0047742691, RGSP = 53.8100363483
The number of Bidders is 18, and the number of Items is 18, truth
seq: RVCG[0] = 12.4359587317, RVCG[n-1] = 6.92892458684
seq: RGSP[0] = 57.292335929, RGSP[n-1] = 47.7861055152
SWVCG = 58.4205908403, SWGSP = 58.4205908403, RVCG = 9.42810418721, RGSP = 53.682537683

seq: RVCG[0] = 1.94486618788, RVCG[n-1] = 2.88465849553
seq: RGSP[0] = 10.1573630584, RGSP[n-1] = 18.829228593
SWVCG = 38.6007925185, SWGSP = 38.6007925185, RVCG = 2.50432235293, RGSP = 18.3543486429
The number of Bidders is 13, and the number of Items is 13, truth
seq: RVCG[0] = 2.63422628135, RVCG[n-1] = 2.38992498795
seq: RGSP[0] = 29.7738377572, RGSP[n-1] = 38.1284222518
SWVCG = 41.8946152007, SWGSP = 41.8946152007, RVCG = 5.60317346121, RGSP = 37.2392522196
The number of Bidders is 14, and the number of Items is 14, random
seq: RVCG[0] = 2.64491884372, RVCG[n-1] = 2.26224536492
seq: RGSP[0] = 16.9235591733, RGSP[n-1] = 18.9825941568
SWVCG = 41.7666029879, SWGSP = 41.7666029879, RVCG = 2.83489448078, RGSP = 20.1610515663
The number of Bidders is 14, and the number of Items is 14, truth
seq: RVCG[0] = 10.9262522853, RVCG[n-1] = 6.10449440907
seq: RGSP[0] = 51.4728348794, RGSP[n-1] = 41.7567530395
SWVCG = 45.0183249195, SWGSP = 45.0183249195, RVCG = 6.38104852384, RGSP = 40.368766883